In [1]:
!pip3 install networkx
!pip3 install xgboost
!pip3 install gensim
!pip install fuzzywuzzy
!pip3 install python-Levenshtein

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 166.7 MB 13 kB/s /s eta 0:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 23.9 MB 18.6 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 11.3 MB/s eta 0:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 50 kB 5.3 MB/s  eta 0:00:01
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp36-cp36m-linux_x86_64.whl size=155944 sha256=7b0aa63f7a06a2efe5bf78b41e92632c69f085662b3a500fcf33dafe7d165d54
  Stored in directory: /hom

In [2]:
!pip3 install nltk

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


## Graph features

In [3]:
import csv
import networkx as nx
import numpy as np
from random import randint
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss

In [4]:
# Create a directed graph
G = nx.read_weighted_edgelist('edgelist.txt', delimiter=' ', create_using=nx.DiGraph(), nodetype=int)
nodes = list(G.nodes())
n = G.number_of_nodes()
m = G.number_of_edges()
print('Number of nodes:', n)
print('Number of edges:', m)


Number of nodes: 33225
Number of edges: 532655


In [5]:
# Create the training matrix. Each row corresponds to a pair of nodes and
# its class label is 1 if it corresponds to an edge and 0, otherwise.
# Use the following 4 features for each pair of nodes:
# (1) in-degree of source node
# (2) out-degree of source node
# (3) in-degree of target node
# (4) out-degree of target node
X_train = np.zeros((2*m, 4))
y_train = np.zeros(2*m)
for i,edge in enumerate(G.edges()):
    # an edge
    X_train[2*i,0] = G.in_degree(edge[0])
    X_train[2*i,1] = G.out_degree(edge[0])
    X_train[2*i,2] = G.in_degree(edge[1])
    X_train[2*i,3] = G.out_degree(edge[1])
    y_train[2*i] = 1

    # a randomly generated pair of nodes
    n1 = nodes[randint(0, n-1)]
    n2 = nodes[randint(0, n-1)]
    X_train[2*i+1,0] = G.in_degree(n1)
    X_train[2*i+1,1] = G.out_degree(n1)
    X_train[2*i+1,2] = G.in_degree(n2)
    X_train[2*i+1,3] = G.out_degree(n2)
    y_train[2*i+1] = 0



In [6]:
# Read test data. Each sample is a pair of nodes
node_pairs = list()
with open('test.txt', 'r') as f:
    for line in f:
        t = line.split(',')
        node_pairs.append((int(t[0]), int(t[1])))

# Create the test matrix. Use the same 4 features as above
X_test = np.zeros((len(node_pairs), 4))
for i,node_pair in enumerate(node_pairs):
    X_test[i,0] = G.in_degree(node_pair[0])
    X_test[i,1] = G.out_degree(node_pair[0])
    X_test[i,2] = G.in_degree(node_pair[1])
    X_test[i,3] = G.out_degree(node_pair[1])



In [7]:
#from sklearn.metrics import log_loss

In [8]:
# Use logistic regression to predict if two nodes are linked by an edge
clf = LogisticRegression()
clf.fit(X_train, y_train)
y_pred = clf.predict_proba(X_test)

In [11]:
clf.score(X_train, y_train)

0.8421342144540087

In [8]:
# Write predictions to a file
y_pred = y_pred[:,1]
predictions = zip(range(len(y_pred)), y_pred)
with open("submission.csv","w") as pred:
    csv_out = csv.writer(pred)
    csv_out.writerow(['id','predicted'])
    for row in predictions:
        csv_out.writerow(row) 

## Try some other models

In [12]:
#!pip3 install xgboost

In [6]:
#alternative models
import xgboost as xgb
from sklearn.metrics import log_loss
# Use XGBoost to predict if two nodes are linked by an edge
clf = xgb.XGBClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict_proba(X_test)
clf.score(X_train, y_train)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[05:36:31] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.9114670846983507

In [18]:
# Write predictions to a file
y_pred = y_pred[:,1]
predictions = zip(range(len(y_pred)), y_pred)
with open("submission.csv","w") as pred:
    csv_out = csv.writer(pred)
    csv_out.writerow(['id','predicted'])
    for row in predictions:
        csv_out.writerow(row)

## Text features

In [7]:
import networkx as nx
import csv
import numpy as np
from random import randint
from sklearn.linear_model import LogisticRegression

In [8]:
# Create a directed graph
G = nx.read_weighted_edgelist('edgelist.txt', delimiter=' ', create_using=nx.DiGraph(), nodetype=int)
m = G.number_of_edges()

# Read the textual content of each domain name
text = list()
for i in range(33226):
    with open('node_information/text/'+str(i)+'.txt', 'r', errors='ignore') as f:
        text.append(f.read())

# Map text to set of terms
text = [set(text[i].split()) for i in range(len(text))]

# Create the training matrix. Each row corresponds to a pair of nodes and
# its class label is 1 if it corresponds to an edge and 0, otherwise.
# Use the following 3 features for each pair of nodes:
# (1) number of unique terms of the source node's textual content
# (2) number of unique terms of the target node's textual content
# (3) number of common terms between the textual content of the two nodes
X_train = np.zeros((2*m, 3))
y_train = np.zeros(2*m)
n = G.number_of_nodes()
for i,edge in enumerate(G.edges()):
    # an edge
    X_train[2*i,0] = len(text[edge[0]])
    X_train[2*i,1] = len(text[edge[1]])
    X_train[2*i,2] = len(text[edge[0]].intersection(text[edge[1]]))
    y_train[2*i] = 1

    # a randomly generated pair of nodes
    n1 = randint(0, n-1)
    n2 = randint(0, n-1)
    X_train[2*i+1,0] = len(text[n1])
    X_train[2*i+1,1] = len(text[n2])
    X_train[2*i+1,2] = len(text[n1].intersection(text[n2]))
    y_train[2*i+1] = 0


In [9]:
print('Number of nodes:', n)
print('Number of edges:', m)

Number of nodes: 33225
Number of edges: 532655


In [10]:
# Read test data. Each sample is a pair of nodes
node_pairs = list()
with open('test.txt', 'r') as f:
    for line in f:
        t = line.split(',')
        node_pairs.append((int(t[0]), int(t[1])))

# Create the test matrix. Use the same 4 features as above
X_test = np.zeros((len(node_pairs), 3))
for i,node_pair in enumerate(node_pairs):
    X_test[i,0] = len(text[node_pair[0]])
    X_test[i,1] = len(text[node_pair[1]])
    X_test[i,2] = len(text[node_pair[0]].intersection(text[node_pair[1]]))

In [18]:
# Use logistic regression to predict if two nodes are linked by an edge
clf = LogisticRegression()
clf.fit(X_train, y_train)
y_pred = clf.predict_proba(X_test)
clf.score(X_train, y_train)

0.6532915301649285

In [18]:
#!pip3 install xgboost

In [19]:
#alternative models
import xgboost as xgb
# Use XGBoost to predict if two nodes are linked by an edge
clf = xgb.XGBClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict_proba(X_test)
clf.score(X_train, y_train)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:03:28] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.7735034872478433

In [ ]:
# Write predictions to a file
y_pred = y_pred[:,1]
predictions = zip(range(len(y_pred)), y_pred)
with open("submission.csv","w") as pred:
    csv_out = csv.writer(pred)
    csv_out.writerow(['id','predicted'])
    for row in predictions:
        csv_out.writerow(row) 

## combine graph & text 

In [11]:
import csv
import networkx as nx
import numpy as np
from random import randint
from sklearn.linear_model import LogisticRegression

# Create a directed graph
G = nx.read_weighted_edgelist('edgelist.txt', delimiter=' ', create_using=nx.DiGraph(), nodetype=int)
nodes = list(G.nodes())
n = G.number_of_nodes()
m = G.number_of_edges()
print('Number of nodes:', n)
print('Number of edges:', m)

# Read the textual content of each domain name
text = list()
for i in range(33226):
    with open('node_information/text/'+str(i)+'.txt', 'r', errors='ignore') as f:
        text.append(f.read())

# Map text to set of terms
text = [set(text[i].split()) for i in range(len(text))]

#train
X_train = np.zeros((2*m, 7))
y_train = np.zeros(2*m)
for i,edge in enumerate(G.edges()):
    # an edge
    X_train[2*i,0] = G.in_degree(edge[0])
    X_train[2*i,1] = G.out_degree(edge[0])
    X_train[2*i,2] = G.in_degree(edge[1])
    X_train[2*i,3] = G.out_degree(edge[1])
    X_train[2*i,4] = len(text[edge[0]])
    X_train[2*i,5] = len(text[edge[1]])
    X_train[2*i,6] = len(text[edge[0]].intersection(text[edge[1]]))
    y_train[2*i] = 1

    # a randomly generated pair of nodes
    n1 = nodes[randint(0, n-1)]
    n2 = nodes[randint(0, n-1)]
    X_train[2*i+1,0] = G.in_degree(n1)
    X_train[2*i+1,1] = G.out_degree(n1)
    X_train[2*i+1,2] = G.in_degree(n2)
    X_train[2*i+1,3] = G.out_degree(n2)
    X_train[2*i+1,4] = len(text[n1])
    X_train[2*i+1,5] = len(text[n2])
    X_train[2*i+1,6] = len(text[n1].intersection(text[n2]))
    y_train[2*i+1] = 0


Number of nodes: 33225
Number of edges: 532655


In [12]:
# Read test data. Each sample is a pair of nodes
node_pairs = list()
with open('test.txt', 'r') as f:
    for line in f:
        t = line.split(',')
        node_pairs.append((int(t[0]), int(t[1])))

# Create the test matrix. Use the same 7 features as above
X_test = np.zeros((len(node_pairs), 7))
for i,node_pair in enumerate(node_pairs):
    X_test[i,0] = G.in_degree(node_pair[0])
    X_test[i,1] = G.out_degree(node_pair[0])
    X_test[i,2] = G.in_degree(node_pair[1])
    X_test[i,3] = G.out_degree(node_pair[1])
    X_test[i,4] = len(text[node_pair[0]])
    X_test[i,5] = len(text[node_pair[1]])
    X_test[i,6] = len(text[node_pair[0]].intersection(text[node_pair[1]]))


In [23]:
# Use logistic regression to predict if two nodes are linked by an edge
clf = LogisticRegression()
clf.fit(X_train, y_train)
y_pred = clf.predict_proba(X_test)
clf.score(X_train, y_train)

0.8751781171677727

In [1]:
#!pip install xgboost

     |████████████████████████████████| 166.7 MB 12 kB/s s eta 0:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [24]:
#alternative models
import xgboost as xgb
# Use XGBoost to predict if two nodes are linked by an edge
clf = xgb.XGBClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict_proba(X_test)
clf.score(X_train, y_train)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:08:03] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.9301339516197163

In [5]:
# Write predictions to a file
y_pred = y_pred[:,1]
predictions = zip(range(len(y_pred)), y_pred)
with open("submission.csv","w") as pred:
    csv_out = csv.writer(pred)
    csv_out.writerow(['id','predicted'])
    for row in predictions:
        csv_out.writerow(row)

## Combined text vectorization doc2vec + Graph + Deep Walk + Text

In [9]:
#import libraries
import networkx as nx
import csv
import numpy as np
import urllib.request
from random import randint
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
#Combined Graph, deepwalk & text
import csv
import networkx as nx
import numpy as np
import urllib.request
from random import randint
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from random import choice
from deepwalk import deepwalk

In [2]:
#use undirected graph (see below)
#create an undirected graph G_ for the deepwalk embeddings
G_ = nx.read_edgelist('edgelist.txt', delimiter=' ', create_using=nx.Graph(), nodetype=int)
nodes_ = list(G_.nodes())
edges_ = list(G_.edges())
n_ = G_.number_of_nodes()
m_ = G_.number_of_edges()
print('Number of nodes:', n_)
print('Number of edges:', m_)

Number of nodes: 33225
Number of edges: 506748


In [3]:
# Create a directed graph for in_degrees & out_degrees
#G = nx.read_edgelist('edgelist.txt', delimiter=' ', create_using=nx.Graph(), nodetype=int)
G = nx.read_weighted_edgelist('edgelist.txt', delimiter=' ', create_using=nx.DiGraph(), nodetype=int)
nodes = list(G.nodes())
edges = list(G.edges())
n = G.number_of_nodes()
m = G.number_of_edges()
print('Number of nodes:', n)
print('Number of edges:', m)


Number of nodes: 33225
Number of edges: 532655


In [4]:
# Read the text of each website
text = list()
node_list = list()
for i in range(33226):
     with open('node_information/text/'+str(i)+'.txt', 'r', errors='ignore') as f:
        text.append(f.read())
        node_list.append(i)
        

In [5]:
#combine the nodes and the texts into a dictionary for embedding
text_dict = dict(zip(node_list, text))

In [6]:
documents = [TaggedDocument(doc, [node]) for node, doc in text_dict.items()]

In [11]:
model = Doc2Vec(documents, vector_size=32, window=10, dm=0, hs=1, min_count=20, workers=8)

In [10]:
# You can obtain the embedding of the abstact of a node using: model.docvecs[node]
E = np.zeros((n_+1, 32))
for node in nodes:
    E[node,:] = model.docvecs[node]

np.save('doc_vectors', E)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:4: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).


In [ ]:
# Read the textual content of each domain name
text = list()
for i in range(33226):
    with open('node_information/text/'+str(i)+'.txt', 'r', errors='ignore') as f:
        text.append(f.read())

# Map text to set of terms
text = [set(text[i].split()) for i in range(len(text))]

In [2]:
#generate random walks on the undirected graph
# Extracts a set of random walks from the network and feeds them to the Skipgram model
n_dim = 64
n_walks = 5
walk_length = 10
model = deepwalk(G_, n_walks, walk_length, n_dim) 

embeddings = np.zeros((n+1, n_dim))
for node_ in G_.nodes():
    embeddings[node_,:] = model.wv[str(node_)]

NameError: name 'deepwalk' is not defined

In [ ]:
# Read the textual content of each domain name
text = list()
for i in range(33226):
    with open('node_information/text/'+str(i)+'.txt', 'r', errors='ignore') as f:
        text.append(f.read())

# Map text to set of terms
text = [set(text[i].split()) for i in range(len(text))]

#train
X_train = np.zeros((2*m, 15))
y_train = np.zeros(2*m)
for i,edge in enumerate(G.edges()):
    # an edge
    X_train[2*i,0] = G.in_degree(edge[0]) #in degree source node
    X_train[2*i,1] = G.out_degree(edge[0]) #out degree esource node
    X_train[2*i,2] = G.in_degree(edge[1]) #in degree target node
    X_train[2*i,3] = G.out_degree(edge[1]) #out degree target node 
    X_train[2*i,4] = len(text[edge[0]]) #number of unique terms of the source node's textual content
    X_train[2*i,5] = len(text[edge[1]]) #number of unique terms of the target node's textual content
    X_train[2*i,6] = len(text[edge[0]].intersection(text[edge[1]])) #number of common terms between the textual content of the two nodes
    X_train[2*i,7] = G_.degree(edge[0]) + G_.degree(edge[1]) #sum of degrees of two nodes
    X_train[2*i,8] = abs(G_.degree(edge[0]) - G_.degree(edge[1])) #abs diff of degrees of two nodes 
    X_train[2*i,9] = np.dot(embeddings[edge[0],:], embeddings[edge[1],:])/(np.linalg.norm(embeddings[edge[0],:])*np.linalg.norm(embeddings[edge[1],:])) #cosine similarity of embeddings of two nodes
    X_train[2*i,10] = len(text_dict[edge[0]]) + len(text_dict[edge[1]]) #sum unique terms of two nodes of pages
    X_train[2*i,11] = abs(len(text_dict[edge[0]]) - len(text_dict[edge[1]])) #abs val of diff of nb unique terms of the two nodes' pages
    X_train[2*i,12] = len(text_dict[edge[0]].intersection(text_dict[edge[1]])) #nb of common terms betweeen pages of the two nodes
    X_train[2*i,13] = np.dot(E[edge[0],:], E[edge[1],:])/(np.linalg.norm(E[edge[0],:])*np.linalg.norm(E[edge[1],:])) #cosine similarity of embeddings of pages
    X_train[2*i,14] = np.linalg.norm(E[edge[0],:]-E[edge[1],:])    #euclidian distance of embeddings of pages
    y_train[2*i] = 1

    # a randomly generated pair of nodes
    n1 = nodes[randint(0, n-1)]
    n2 = nodes[randint(0, n-1)]
    X_train[2*i+1,0] = G.in_degree(n1) #in degree source node
    X_train[2*i+1,1] = G.out_degree(n1)  #out degree esource node
    X_train[2*i+1,2] = G.in_degree(n2) #in degree target node
    X_train[2*i+1,3] = G.out_degree(n2) #out degree target nod
    X_train[2*i+1,4] = len(text[n1])  #number of unique terms of the source node's textual content
    X_train[2*i+1,5] = len(text[n2]) #number of unique terms of the target node's textual content
    X_train[2*i+1,6] = len(text[n1].intersection(text[n2])) #number of common terms between the textual content of the two nodes
    X_train[2*i+1,7] = G_.degree(n1) + G_.degree(n2)  #sum of degrees of two nodes
    X_train[2*i+1,8] = abs(G_.degree(n1) - G_.degree(n2)) #abs diff of degrees of two nodes 
    X_train[2*i+1,9] = np.dot(embeddings[n1,:], embeddings[n2,:])/(np.linalg.norm(embeddings[n1,:])*np.linalg.norm(embeddings[n2,:])) #cosine similarity of embeddings of two nodes
    X_train[2*i+1,10] = len(text_dict[n1]) + len(text_dict[n2]) #sum unique terms of two nodes of pages
    X_train[2*i+1,11] = abs(len(text_dict[n1]) - len(text_dict[n2])) #abs val of diff of nb unique terms of the two nodes' pages
    X_train[2*i+1,12] = len(text_dict[n1].intersection(text_dict[n2])) #nb of common terms betweeen pages of the two nodes
    X_train[2*i+1,13] = np.dot(E[n1,:], E[n2,:])/(np.linalg.norm(E[n1,:])*np.linalg.norm(E[n2,:])) #cosine similarity of embeddings of pages
    X_train[2*i+1,14] = np.linalg.norm(E[n1,:]-E[n2,:]) #euclidian distance of embeddings of pages
    y_train[2*i+1] = 0

In [ ]:
# Read test data. Each sample is a pair of nodes
node_pairs = list()
with open('test.txt', 'r') as f:
    for line in f:
        t = line.split(',')
        node_pairs.append((int(t[0]), int(t[1])))

# Create the test matrix. Use the same 15 features as above
X_test = np.zeros((len(node_pairs), 15))
for i,node_pair in enumerate(node_pairs):
    X_test[i,0] = G.in_degree(node_pair[0])
    X_test[i,1] = G.out_degree(node_pair[0])
    X_test[i,2] = G.in_degree(node_pair[1])
    X_test[i,3] = G.out_degree(node_pair[1])
    X_test[i,4] = len(text[node_pair[0]])
    X_test[i,5] = len(text[node_pair[1]])
    X_test[i,6] = len(text[node_pair[0]].intersection(text[node_pair[1]]))
    X_test[i,7] = G_.degree(node_pair[0]) + G_.degree(node_pair[1])
    X_test[i,8] = abs(G_.degree(node_pair[0]) - G_.degree(node_pair[1]))
    X_test[i,9] = np.dot(embeddings[node_pair[0],:], embeddings[node_pair[1],:])/(np.linalg.norm(embeddings[node_pair[0],:])*np.linalg.norm(embeddings[node_pair[1],:]))
    X_test[i,10] = len(text_dict[node_pair[0]]) + len(text_dict[node_pair[1]])
    X_test[i,11] = abs(len(text_dict[node_pair[0]]) - len(text_dict[node_pair[1]]))
    X_test[i,12] = len(text_dict[node_pair[0]].intersection(text_dict[node_pair[1]]))
    X_test[i,13] = np.dot(E[node_pair[0],:], E[node_pair[1],:])/(np.linalg.norm(E[node_pair[0],:])*np.linalg.norm(E[node_pair[1],:]))
    X_test[i,14] = np.linalg.norm(E[node_pair[0],:]-E[node_pair[1],:])
    

In [ ]:
# Use logistic regression to predict if two nodes are linked by an edge
clf = LogisticRegression()
clf.fit(X_train, y_train)
y_pred = clf.predict_proba(X_test)
clf.score(X_train, y_train)

In [ ]:
#alternative models
import xgboost as xgb
# Use XGBoost to predict if two nodes are linked by an edge
clf = xgb.XGBClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict_proba(X_test)
clf.score(X_train, y_train)

In [ ]:
# Write predictions to a file
y_pred = y_pred[:,1]
predictions = zip(range(len(y_pred)), y_pred)
with open("submission.csv","w") as pred:
    csv_out = csv.writer(pred)
    csv_out.writerow(['id','predicted'])
    for row in predictions:
        csv_out.writerow(row)

## Stopwords Removal

In [ ]:
#stopwords removal french 

In [13]:
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import string
from random import choice

In [29]:
import nltk
nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [Errno 104]
[nltk_data]     Connection reset by peer>


False

In [12]:
stopwords = stopwords.words('french')
print(stopwords)

<WordListCorpusReader in '.../corpora/stopwords' (not loaded yet)>

In [15]:
stopwords=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

In [ ]:
french_stopwords = set(stopwords.words('french'))

In [18]:
text = list()
pages = dict()
list_pages = list()
punctuation = set(string.punctuation)
counter = 0
for i in range(33226):
    with open('node_information/text/'+str(i)+'.txt', 'r', errors='ignore') as f:
        text.append(f.read())

        for idx, line in enumerate(text):
        node, page = idx, line
        processed_abstract = ''.join([w if w not in punctuation else ' ' for w in abstract]) #.lower()
    abstract = [word for word in processed_abstract.split() if word not in stopwords]
    abstracts[int(node)] = abstract
    list_abstracts.append(abstract)
    counter += 1
       
    
    
    
    
    # Read the abstract of each paper
abstracts = dict()
list_abstracts = list()
punctuation = set(string.punctuation)
counter = 0
for idx, line in enumerate(text):
    node, abstract = idx, line
    processed_abstract = ''.join([w if w not in punctuation else ' ' for w in abstract]) #.lower()
    abstract = [word for word in processed_abstract.split() if word not in stopwords]
    abstracts[int(node)] = abstract
    list_abstracts.append(abstract)
    counter += 1

# Map text to set of terms
for node in abstracts:
    abstracts[node] = set(abstracts[node])
    

## Code from previous challenge

In [ ]:
# Read the abstract of each paper
abstracts = dict()
list_abstracts = list()
punctuation = set(string.punctuation)
counter = 0
for idx, line in enumerate(text):
    node, abstract = idx, line
    processed_abstract = ''.join([w if w not in punctuation else ' ' for w in abstract]) #.lower()
    abstract = [word for word in processed_abstract.split() if word not in stopwords]
    abstracts[int(node)] = abstract
    list_abstracts.append(abstract)
    counter += 1

# Map text to set of terms
for node in abstracts:
    abstracts[node] = set(abstracts[node])
    

In [16]:
print(len(abstracts), counter, n)

33226 33226 33225


In [27]:
#!pip3 install gensim

In [17]:
# WORD2VEC
from gensim.models import Word2Vec

model_w2v = Word2Vec(vector_size=64, window=5, min_count=0, sg=1, workers=8)
model_w2v.build_vocab(list_abstracts)


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [18]:
model_w2v.train(list_abstracts, total_examples=model_w2v.corpus_count, epochs=5) 

(1274833, 1280935)

In [19]:
n_dim = 64
coutner =0
embeddings_abstracts = np.zeros((n, n_dim))
pass_nodes = list()
for node in nodes:
  if len(abstracts[int(node)]) > 1:
    embeddings_abstracts[node,:] = model_w2v.wv[abstracts[int(node)]].mean(0)
  else:
    pass_nodes.append(node)
    pass


In [20]:
print(pass_nodes)

[20421, 14319, 6807, 3409, 8019, 6108, 29573, 3753, 2997, 3855, 6799, 19735, 8523, 501, 22055, 26112, 5381, 4545, 3959, 1634, 5437, 5537, 21641, 5540, 18300, 5562, 5548, 5530, 5542, 5547, 5538, 5554, 5543, 6277, 347, 341, 6033, 402, 5944, 8531, 7659, 4829, 10715, 19512, 8640, 8082, 6055, 5622, 3841, 20288, 3304, 20287, 20294, 6813, 18672, 14734, 5620, 15752, 17443, 15494, 4927, 3764, 20271, 85, 9985, 6734, 20309, 20268, 19931, 213, 20282, 14667, 10232, 3079, 3134, 6397, 9071, 1770, 8118, 873, 639, 6567, 13039, 5447, 9830, 5461, 4528, 9912, 17593, 6048, 225, 343, 11480, 3567, 7282, 8964, 5034, 908, 11310, 7300, 503, 8405, 33119, 1867, 30078, 8716, 1866, 6708, 5761, 9461, 11802, 3444, 1873, 20250, 17186, 13992, 26923, 20796, 9773, 3448, 6487, 2888, 6616, 5067, 9005, 5843, 14061, 23603, 25701, 19579, 6107, 5929, 19706, 1868, 18326, 9965, 4458, 20164, 12003, 6433, 14614, 12294, 11431, 13203, 14645, 602, 18158, 25558, 4947, 18149, 13111, 14009, 5091, 13325, 1077, 11060, 13434, 14300, 31730,

In [7]:
# DEEPWALK
############## Task 1
# Simulates a random walk of length "walk_length" starting from node "node"
def random_walk(G, node, walk_length):
	walk = [node]
	
	for i in range(walk_length-1):
		nbrs = list(G.neighbors(walk[-1]))
		current_node = choice(nbrs)
		walk.append(current_node)

	walk = [str(node) for node in walk]
	return walk


############## Task 2
# Runs "num_walks" random walks from each node
def generate_walks(G, num_walks, walk_length):
	walks = []
	
	nodes = list(G.nodes())
	for i in range(num_walks):
		idx = np.random.permutation(len(nodes))
		for j in range(len(nodes)):
			node = nodes[idx[j]]
			walk = random_walk(G, node, walk_length)
			walks.append(walk)

	return walks

# Simulates walks and uses the Skipgram model to learn node representations
def deepwalk(G, num_walks, walk_length, n_dim):
    print("Generating walks")
    walks = generate_walks(G, num_walks, walk_length)

    print("Training word2vec")
    model = Word2Vec(size=n_dim, window=8, min_count=0, sg=1, workers=8)
    model.build_vocab(walks)
    model.train(walks, total_examples=model.corpus_count, epochs=5)

    return model

In [ ]:
n_dim = 64
n_walks = 5
walk_length = 10
model_dw = deepwalk(G, n_walks, walk_length, n_dim) 

In [2]:
#!pip3 install python-Levenshtein

     |████████████████████████████████| 50 kB 7.9 MB/s  eta 0:00:01
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp36-cp36m-linux_x86_64.whl size=155937 sha256=950892b84d8d98013ad05b242a70e861e004196396dc7f5c541534fb555ee0c2
  Stored in directory: /home/ec2-user/.cache/pip/wheels/4a/a4/bf/d761b0899395c75fa76d003d607b3869ee47f5035b8afc30a2
Successfully built python-Levenshtein
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


## Combined Graph, Deepwalk & text features

In [1]:
#Combined Graph, deepwalk & text
import csv
import networkx as nx
import numpy as np
import urllib.request
from random import randint
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from random import choice
from deepwalk import deepwalk

In [13]:
# Create a directed graph for in_degrees & out_degrees
#G = nx.read_edgelist('edgelist.txt', delimiter=' ', create_using=nx.Graph(), nodetype=int)
G = nx.read_weighted_edgelist('edgelist.txt', delimiter=' ', create_using=nx.DiGraph(), nodetype=int)
nodes = list(G.nodes())
edges = list(G.edges())
n = G.number_of_nodes()
m = G.number_of_edges()
print('Number of nodes:', n)
print('Number of edges:', m)


Number of nodes: 33225
Number of edges: 532655


In [15]:
#create an undirected graph G_ for the deepwalk embeddings
G_ = nx.read_edgelist('edgelist.txt', delimiter=' ', create_using=nx.Graph(), nodetype=int)
nodes_ = list(G_.nodes())
edges_ = list(G_.edges())
n_ = G_.number_of_nodes()
m_ = G_.number_of_edges()
print('Number of nodes:', n_)
print('Number of edges:', m_)

Number of nodes: 33225
Number of edges: 506748


In [16]:
#generate random walks on the undirected graph
# Extracts a set of random walks from the network and feeds them to the Skipgram model
n_dim = 64
n_walks = 5
walk_length = 10
model = deepwalk(G_, n_walks, walk_length, n_dim) 

embeddings = np.zeros((n+1, n_dim))
for node_ in G_.nodes():
    embeddings[node_,:] = model.wv[str(node_)]

Generating walks
Training word2vec


In [11]:
len(embeddings)

33226

In [17]:
# Read the textual content of each domain name
text = list()
for i in range(33226):
    with open('node_information/text/'+str(i)+'.txt', 'r', errors='ignore') as f:
        text.append(f.read())

# Map text to set of terms
text = [set(text[i].split()) for i in range(len(text))]

#train
X_train = np.zeros((2*m, 10))
y_train = np.zeros(2*m)
for i,edge in enumerate(G.edges()):
    # an edge
    X_train[2*i,0] = G.in_degree(edge[0])
    X_train[2*i,1] = G.out_degree(edge[0])
    X_train[2*i,2] = G.in_degree(edge[1])
    X_train[2*i,3] = G.out_degree(edge[1])
    X_train[2*i,4] = len(text[edge[0]])
    X_train[2*i,5] = len(text[edge[1]])
    X_train[2*i,6] = len(text[edge[0]].intersection(text[edge[1]]))
    X_train[2*i,7] = G_.degree(edge[0]) + G_.degree(edge[1])
    X_train[2*i,8] = abs(G_.degree(edge[0]) - G_.degree(edge[1]))
    X_train[2*i,9] = np.dot(embeddings[edge[0],:], embeddings[edge[1],:])/(np.linalg.norm(embeddings[edge[0],:])*np.linalg.norm(embeddings[edge[1],:]))
    y_train[2*i] = 1

    # a randomly generated pair of nodes
    n1 = nodes[randint(0, n-1)]
    n2 = nodes[randint(0, n-1)]
    X_train[2*i+1,0] = G.in_degree(n1)
    X_train[2*i+1,1] = G.out_degree(n1)
    X_train[2*i+1,2] = G.in_degree(n2)
    X_train[2*i+1,3] = G.out_degree(n2)
    X_train[2*i+1,4] = len(text[n1])
    X_train[2*i+1,5] = len(text[n2])
    X_train[2*i+1,6] = len(text[n1].intersection(text[n2]))
    X_train[2*i+1,7] = G_.degree(n1) + G_.degree(n2)
    X_train[2*i+1,8] = abs(G_.degree(n1) - G_.degree(n2))
    X_train[2*i+1,9] = np.dot(embeddings[n1,:], embeddings[n2,:])/(np.linalg.norm(embeddings[n1,:])*np.linalg.norm(embeddings[n2,:]))
    y_train[2*i+1] = 0

In [18]:
# Read test data. Each sample is a pair of nodes
node_pairs = list()
with open('test.txt', 'r') as f:
    for line in f:
        t = line.split(',')
        node_pairs.append((int(t[0]), int(t[1])))

# Create the test matrix. Use the same 7 features as above
X_test = np.zeros((len(node_pairs), 10))
for i,node_pair in enumerate(node_pairs):
    X_test[i,0] = G.in_degree(node_pair[0])
    X_test[i,1] = G.out_degree(node_pair[0])
    X_test[i,2] = G.in_degree(node_pair[1])
    X_test[i,3] = G.out_degree(node_pair[1])
    X_test[i,4] = len(text[node_pair[0]])
    X_test[i,5] = len(text[node_pair[1]])
    X_test[i,6] = len(text[node_pair[0]].intersection(text[node_pair[1]]))
    X_test[i,7] = G_.degree(node_pair[0]) + G_.degree(node_pair[1])
    X_test[i,8] = abs(G_.degree(node_pair[0]) - G_.degree(node_pair[1]))
    X_test[i,9] = np.dot(embeddings[node_pair[0],:], embeddings[node_pair[1],:])/(np.linalg.norm(embeddings[node_pair[0],:])*np.linalg.norm(embeddings[node_pair[1],:]))

In [19]:
# Use logistic regression to predict if two nodes are linked by an edge
clf = LogisticRegression()
clf.fit(X_train, y_train)
y_pred = clf.predict_proba(X_test)
clf.score(X_train, y_train)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.8771249683190808

In [20]:
#alternative models
import xgboost as xgb
# Use XGBoost to predict if two nodes are linked by an edge
clf = xgb.XGBClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict_proba(X_test)
clf.score(X_train, y_train)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:25:30] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.9697740563779557

In [21]:
# Write predictions to a file
y_pred = y_pred[:,1]
predictions = zip(range(len(y_pred)), y_pred)
with open("submission.csv","w") as pred:
    csv_out = csv.writer(pred)
    csv_out.writerow(['id','predicted'])
    for row in predictions:
        csv_out.writerow(row) 

## 0.9697740563779557 Graph, Deepwalk, & text in XGBoost - 0.16427 

## Extra deepwalk features (common terms & euclidian distance)

In [1]:
#Combined Graph, deepwalk & text
import csv
import networkx as nx
import numpy as np
import urllib.request
from random import randint
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from random import choice
from deepwalk import deepwalk

In [2]:
# Create a directed graph for in_degrees & out_degrees
#G = nx.read_edgelist('edgelist.txt', delimiter=' ', create_using=nx.Graph(), nodetype=int)
G = nx.read_weighted_edgelist('edgelist.txt', delimiter=' ', create_using=nx.DiGraph(), nodetype=int)
nodes = list(G.nodes())
edges = list(G.edges())
n = G.number_of_nodes()
m = G.number_of_edges()
print('Number of nodes:', n)
print('Number of edges:', m)


Number of nodes: 33225
Number of edges: 532655


In [3]:
#create an undirected graph G_ for the deepwalk embeddings
G_ = nx.read_edgelist('edgelist.txt', delimiter=' ', create_using=nx.Graph(), nodetype=int)
nodes_ = list(G_.nodes())
edges_ = list(G_.edges())
n_ = G_.number_of_nodes()
m_ = G_.number_of_edges()
print('Number of nodes:', n_)
print('Number of edges:', m_)

Number of nodes: 33225
Number of edges: 506748


In [4]:
#generate random walks on the undirected graph
# Extracts a set of random walks from the network and feeds them to the Skipgram model
n_dim = 64
n_walks = 5
walk_length = 10
model = deepwalk(G_, n_walks, walk_length, n_dim) 

embeddings = np.zeros((n+1, n_dim))
for node_ in G_.nodes():
    embeddings[node_,:] = model.wv[str(node_)]

Generating walks
Training word2vec


In [5]:
# Read the textual content of each domain name
text = list()
for i in range(33226):
    with open('node_information/text/'+str(i)+'.txt', 'r', errors='ignore') as f:
        text.append(f.read())

# Map text to set of terms
text = [set(text[i].split()) for i in range(len(text))]

#train
X_train = np.zeros((2*m, 11))
y_train = np.zeros(2*m)
for i,edge in enumerate(G.edges()):
    # an edge
    X_train[2*i,0] = G.in_degree(edge[0]) #in degree source node
    X_train[2*i,1] = G.out_degree(edge[0]) #out degree esource node
    X_train[2*i,2] = G.in_degree(edge[1]) #in degree target node
    X_train[2*i,3] = G.out_degree(edge[1]) #out degree target node 
    X_train[2*i,4] = len(text[edge[0]]) #number of unique terms of the source node's textual content
    X_train[2*i,5] = len(text[edge[1]]) #number of unique terms of the target node's textual content
    X_train[2*i,6] = len(text[edge[0]].intersection(text[edge[1]])) #number of common terms between the textual content of the two nodes
    X_train[2*i,7] = G_.degree(edge[0]) + G_.degree(edge[1]) #sum of degrees of two nodes
    X_train[2*i,8] = abs(G_.degree(edge[0]) - G_.degree(edge[1])) #abs diff of degrees of two nodes 
    X_train[2*i,9] = np.dot(embeddings[edge[0],:], embeddings[edge[1],:])/(np.linalg.norm(embeddings[edge[0],:])*np.linalg.norm(embeddings[edge[1],:])) #cosine similarity of embeddings of two nodes
    #X_train[2*i,10] = len(embeddings[edge[0]].intersection(embeddings[edge[1]])) #nb of common terms betweeen embeddings of pages of the two nodes
    X_train[2*i,10] = np.linalg.norm(embeddings[edge[0],:]-embeddings[edge[1],:])    #euclidian distance of embeddings of pages
    y_train[2*i] = 1

    # a randomly generated pair of nodes
    n1 = nodes[randint(0, n-1)]
    n2 = nodes[randint(0, n-1)]
    X_train[2*i+1,0] = G.in_degree(n1) #in degree source node
    X_train[2*i+1,1] = G.out_degree(n1)  #out degree esource node
    X_train[2*i+1,2] = G.in_degree(n2) #in degree target node
    X_train[2*i+1,3] = G.out_degree(n2) #out degree target nod
    X_train[2*i+1,4] = len(text[n1])  #number of unique terms of the source node's textual content
    X_train[2*i+1,5] = len(text[n2]) #number of unique terms of the target node's textual content
    X_train[2*i+1,6] = len(text[n1].intersection(text[n2])) #number of common terms between the textual content of the two nodes
    X_train[2*i+1,7] = G_.degree(n1) + G_.degree(n2)  #sum of degrees of two nodes
    X_train[2*i+1,8] = abs(G_.degree(n1) - G_.degree(n2)) #abs diff of degrees of two nodes 
    X_train[2*i+1,9] = np.dot(embeddings[n1,:], embeddings[n2,:])/(np.linalg.norm(embeddings[n1,:])*np.linalg.norm(embeddings[n2,:])) #cosine similarity of embeddings of two nodes
    #X_train[2*i+1,10] = len(embeddings[n1].intersection(embeddings[n2])) #nb of common terms betweeen embeddings of pages of the two nodes
    X_train[2*i+1,10] = np.linalg.norm(embeddings[n1,:]-embeddings[n2,:]) #euclidian distance of embeddings of pages
    y_train[2*i+1] = 0

In [6]:
# Read test data. Each sample is a pair of nodes
node_pairs = list()
with open('test.txt', 'r') as f:
    for line in f:
        t = line.split(',')
        node_pairs.append((int(t[0]), int(t[1])))

# Create the test matrix. Use the same 12 features as above
X_test = np.zeros((len(node_pairs), 11))
for i,node_pair in enumerate(node_pairs):
    X_test[i,0] = G.in_degree(node_pair[0])
    X_test[i,1] = G.out_degree(node_pair[0])
    X_test[i,2] = G.in_degree(node_pair[1])
    X_test[i,3] = G.out_degree(node_pair[1])
    X_test[i,4] = len(text[node_pair[0]])
    X_test[i,5] = len(text[node_pair[1]])
    X_test[i,6] = len(text[node_pair[0]].intersection(text[node_pair[1]]))
    X_test[i,7] = G_.degree(node_pair[0]) + G_.degree(node_pair[1])
    X_test[i,8] = abs(G_.degree(node_pair[0]) - G_.degree(node_pair[1]))
    X_test[i,9] = np.dot(embeddings[node_pair[0],:], embeddings[node_pair[1],:])/(np.linalg.norm(embeddings[node_pair[0],:])*np.linalg.norm(embeddings[node_pair[1],:]))
    #X_test[i,10] = len(embeddings[node_pair[0]].intersection(embeddings[node_pair[1]]))
    X_test[i,10] = np.linalg.norm(embeddings[node_pair[0],:]-embeddings[node_pair[1],:])

In [7]:
# Use logistic regression to predict if two nodes are linked by an edge
clf = LogisticRegression()
clf.fit(X_train, y_train)
y_pred = clf.predict_proba(X_test)
clf.score(X_train, y_train)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.91099210558429

In [8]:
#alternative models
import xgboost as xgb
# Use XGBoost to predict if two nodes are linked by an edge
clf = xgb.XGBClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict_proba(X_test)
clf.score(X_train, y_train)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:10:15] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.9704452225173893

In [9]:
# Write predictions to a file
y_pred = y_pred[:,1]
predictions = zip(range(len(y_pred)), y_pred)
with open("submission.csv","w") as pred:
    csv_out = csv.writer(pred)
    csv_out.writerow(['id','predicted'])
    for row in predictions:
        csv_out.writerow(row) 

## 0.9704452225173893 for xgboost with addition of euclidian distance on deep walks